# Usage of Smart EDGAR as Library to Access EDGAR Filings
The initial concept of Smart EDGAR was to provide the tools to download the data from the EDGAR website and 
- store the information as local files
- store the information in a SQL database
- provide reporting functionality from this data via Webservices

We have extended the funcitonality a little bit so that the Smart EDGAR Library can be used without having any data available locally. So we provide 
- an API which retrieves the information directly from the EDGAR Website.
- Company KPIs from the EDGAR Website or pulling the data from a REST webservice

## Setup
We install the following libraries with the help of Maven
- Smart Edgar
- Jupyter JDK Extensions (to display our custom Tables)

In [23]:
%classpath reset 


Reset done, please restart the kernel.


In [24]:
%classpath config resolver maven-public http://software.pschatzmann.ch/repository/maven-public/
%%classpath add mvn 
ch.pschatzmann:smart-edgar:1.0.2
ch.pschatzmann:jupyter-jdk-extensions:1.0.0

Added new repo: maven-public


In [25]:
import ch.pschatzmann.display._
import ch.pschatzmann.edgar.base._
import ch.pschatzmann.edgar.reporting.company._
import ch.pschatzmann.edgar.dataload.rss.RSSDataSource
import ch.pschatzmann.edgar.utils.Utils
import ch.pschatzmann.edgar.base.Fact._
import ch.pschatzmann.edgar.dataload.online._

Displayers.setup

true

## Access To Filings
We provide an API to retrieve the Company information and the related filings.

We can pass a ticker symbol or a company number to the constructor of the CompanyInformation

In [26]:
%%time
val company = new CompanyInformation("AAPL")

CPU times: user 0 ns, sys: 255 µs, total: 255 µs 
Wall Time: 934 ms



Apple Inc. (0000320193)

Optionally we can indicate the form type in the constructor. If no form is indicated we retrieve all documents but this is taking quite some time.

In [27]:
%%time
val company = new CompanyInformation("320193","10-K")

CPU times: user 0 ns, sys: 211 µs, total: 211 µs 
Wall Time: 624 ms



Apple Inc. (0000320193)

We can list all the available filings

In [28]:
%%time
company.entries

CPU times: user 0 ns, sys: 225 µs, total: 225 µs 
Wall Time: 1 s



[0000320193-19-000119:10-K, 0000320193-18-000145:10-K, 0000320193-17-000070:10-K, 0001628280-16-020309:10-K, 0001193125-15-356351:10-K, 0001193125-14-383437:10-K, 0001193125-13-416534:10-K, 0001193125-12-444068:10-K, 0001193125-11-282113:10-K, 0001193125-10-238044:10-K, 0001193125-10-012091:10-K/A, 0001193125-09-214859:10-K, 0001193125-08-224958:10-K, 0001047469-07-009340:10-K, 0001104659-06-084288:10-K, 0001104659-05-058421:10-K, 0001047469-04-035975:10-K, 0001047469-03-041604:10-K, 0001047469-02-007674:10-K, 0000912057-01-544436:10-K405, 0000912057-00-053623:10-K, 0000912057-99-010244:10-K, 0001047469-98-044981:10-K405, 0001047469-98-001822:10-K/A, 0001047469-97-006960:10-K, 0000320193-96-000023:10-K, 0000320193-95-000016:10-K, 0000320193-94-000016:10-K]

We can also limit the enties with a Regex expression

In [29]:
%%time
var entries = company.entries("10-K")

CPU times: user 0 ns, sys: 278 µs, total: 278 µs 
Wall Time: 366 ms



[0000320193-19-000119:10-K, 0000320193-18-000145:10-K, 0000320193-17-000070:10-K, 0001628280-16-020309:10-K, 0001193125-15-356351:10-K, 0001193125-14-383437:10-K, 0001193125-13-416534:10-K, 0001193125-12-444068:10-K, 0001193125-11-282113:10-K, 0001193125-10-238044:10-K, 0001193125-09-214859:10-K, 0001193125-08-224958:10-K, 0001047469-07-009340:10-K, 0001104659-06-084288:10-K, 0001104659-05-058421:10-K, 0001047469-04-035975:10-K, 0001047469-03-041604:10-K, 0001047469-02-007674:10-K, 0000912057-00-053623:10-K, 0000912057-99-010244:10-K, 0001047469-97-006960:10-K, 0000320193-96-000023:10-K, 0000320193-95-000016:10-K, 0000320193-94-000016:10-K]

We can get the XBRL object which contains the informaiton of all documents

In [30]:
%%time
var xbrl = company.getXBRL
xbrl.isEmpty

CPU times: user 0 ns, sys: 288 µs, total: 288 µs 
Wall Time: 50 s



false

On the level of the filing entry we provide the following:

In [31]:
var entry = entries.get(0)


0000320193-19-000119:10-K

We can e.g. display the Form or title ...

In [32]:
entry.getForm

10-K

In [33]:
entry.getTitle

10-K  - Annual report [Section 13 and 15(d), not S-K Item 405]

In [34]:
entry.getLink

https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/0000320193-19-000119-index.htm

Each link is pointing to a list of detailed documents:

In [35]:
import scala.collection.JavaConverters._

entry.getLinksToDocuments.iterator.asScala.toSeq.foreach(r => println(r))

/Archives/edgar/data/320193/000032019319000119/0000320193-19-000119-index-headers.html
/Archives/edgar/data/320193/000032019319000119/0000320193-19-000119-index.html
/Archives/edgar/data/320193/000032019319000119/0000320193-19-000119.txt
/Archives/edgar/data/320193/000032019319000119/0000320193-19-000119-xbrl.zip
/Archives/edgar/data/320193/000032019319000119/a10-k20199282019.htm
/Archives/edgar/data/320193/000032019319000119/a10-k20199282019_htm.xml
/Archives/edgar/data/320193/000032019319000119/a10-kexhibit10152019.htm
/Archives/edgar/data/320193/000032019319000119/a10-kexhibit10162019.htm
/Archives/edgar/data/320193/000032019319000119/a10-kexhibit2112019.htm
/Archives/edgar/data/320193/000032019319000119/a10-kexhibit2312019.htm
/Archives/edgar/data/320193/000032019319000119/a10-kexhibit3112019.htm
/Archives/edgar/data/320193/000032019319000119/a10-kexhibit3122019.htm
/Archives/edgar/data/320193/000032019319000119/a10-kexhibit3212019.htm
/Archives/edgar/data/320193/000032019319000119

null

We can get the XBRL document link

In [36]:
entry.getLinkToXbrl

Optional[/Archives/edgar/data/320193/000032019319000119/0000320193-19-000119-xbrl.zip]

The base URL for the documents can be determied with

In [37]:
entry.getLinkBase

https://www.sec.gov

Or we can get the XBRL object

In [38]:
%%time
val xbrl = entry.getXBRL.get
xbrl.isEmpty

CPU times: user 0 ns, sys: 217 µs, total: 217 µs 
Wall Time: 2 s



false

In [39]:
val values = xbrl.findValues("NetIncomeLoss")
values.size

23

Finally we display the xbrl values in a table

In [40]:
import scala.collection.JavaConverters._

values.asScala.map(factValue => factValue.getAttributes).asJava

We double check the existence of xbrl documents

In [41]:
import scala.collection.JavaConverters._

new CompanyInformation("320193","10-K").entries().asScala
   .map(e => (e.getForm, e.getAccessionNumber, e.getLinkToXbrl) )
   .foreach(t => println(t) )

(10-K,0000320193-19-000119,Optional[/Archives/edgar/data/320193/000032019319000119/0000320193-19-000119-xbrl.zip])
(10-K,0000320193-18-000145,Optional[/Archives/edgar/data/320193/000032019318000145/0000320193-18-000145-xbrl.zip])
(10-K,0000320193-17-000070,Optional[/Archives/edgar/data/320193/000032019317000070/0000320193-17-000070-xbrl.zip])
(10-K,0001628280-16-020309,Optional[/Archives/edgar/data/320193/000162828016020309/0001628280-16-020309-xbrl.zip])
(10-K,0001193125-15-356351,Optional[/Archives/edgar/data/320193/000119312515356351/0001193125-15-356351-xbrl.zip])
(10-K,0001193125-14-383437,Optional[/Archives/edgar/data/320193/000119312514383437/0001193125-14-383437-xbrl.zip])
(10-K,0001193125-13-416534,Optional[/Archives/edgar/data/320193/000119312513416534/0001193125-13-416534-xbrl.zip])
(10-K,0001193125-12-444068,Optional[/Archives/edgar/data/320193/000119312512444068/0001193125-12-444068-xbrl.zip])
(10-K,0001193125-11-282113,Optional[/Archives/edgar/data/320193/0001193125112821

null

We can retrieve the values from the XBRL filings

In [42]:
new CompanyInformation("320193","10-K")
    .getXBRL.findValues("NetIncomeLoss").asScala
    .map(factValue => factValue.getAttributes)
    .asJava

## Company KPIs Direcly from EDGAR
The following example is providing KPIs directly from EDGAR 

In [43]:
%%time
val values = new CompanyEdgarValuesEdgar(new CompanyInformation("320193"))
    .setFilter(new FilterYearly())
    .setParameterNames("NetIncomeLoss","OperatingIncomeLoss","ResearchAndDevelopmentExpense",
        "CashAndCashEquivalentsAtCarryingValue","AvailableForSaleSecuritiesCurrent","AccountsReceivableNetCurrent",
        "Revenues","SalesRevenueNet","InventoryNet","AssetsCurrent","LiabilitiesCurrent","Assets","EarningsPerShareBasic",
        "StockholdersEquity")
    .addFormula("Revenue","Edgar.coalesce('Revenues', 'SalesRevenueNet')")
    .addFormula("QuickRatio","(CashAndCashEquivalentsAtCarryingValue + AccountsReceivableNetCurrent + AvailableForSaleSecuritiesCurrent) / LiabilitiesCurrent")
    .addFormula("CurrentRatio","AssetsCurrent / LiabilitiesCurrent")
    .addFormula("InventoryTurnover","Revenue / InventoryNet")
    .addFormula("NetProfitMargin","NetIncomeLoss / Revenue")
    .addFormula("SalesResearchRatio%","ResearchAndDevelopmentExpense / Revenue *100")
    .addFormula("NetIncomeResearchRatio%","ResearchAndDevelopmentExpense / NetIncomeLoss * 100")
    .addFormula("NetIncomeChange%", "NetIncomeLoss - Edgar.lag('NetIncomeLoss', -1) / Edgar.lag('NetIncomeLoss', -1) * 100 ")  
    .addFormula("RevenueChange%", "Edgar.percentChange('Revenue')" )  
    .addFormula("ResearchAndDevelopmentChange%","Edgar.percentChange('ResearchAndDevelopmentExpense')" )
    .removeParameterNames("Revenues","SalesRevenueNet")

val table = values.getTable

CPU times: user 0 ns, sys: 234 µs, total: 234 µs 
Wall Time: 8 s



## Company KPIs Direcly via REST
The following example is providing KPI directly via [REST](https://pschatzmann.ch/edgar/index.html#/DBService/post_db_companyValues)

In [44]:
%%time
val values = new CompanyEdgarValuesRest(new CompanyInformation("320193"))
    .setFilter(new FilterYearly())
    .setParameterNames("NetIncomeLoss","OperatingIncomeLoss","ResearchAndDevelopmentExpense",
        "CashAndCashEquivalentsAtCarryingValue","AvailableForSaleSecuritiesCurrent","AccountsReceivableNetCurrent",
        "Revenues","SalesRevenueNet","InventoryNet","AssetsCurrent","LiabilitiesCurrent","Assets","EarningsPerShareBasic",
        "StockholdersEquity")
    .addFormula("Revenue","Edgar.coalesce('Revenues', 'SalesRevenueNet')")
    .addFormula("QuickRatio","(CashAndCashEquivalentsAtCarryingValue + AccountsReceivableNetCurrent + AvailableForSaleSecuritiesCurrent) / LiabilitiesCurrent")
    .addFormula("CurrentRatio","AssetsCurrent / LiabilitiesCurrent")
    .addFormula("InventoryTurnover","Revenue / InventoryNet")
    .addFormula("NetProfitMargin","NetIncomeLoss / Revenue")
    .addFormula("SalesResearchRatio%","ResearchAndDevelopmentExpense / Revenue *100")
    .addFormula("NetIncomeResearchRatio%","ResearchAndDevelopmentExpense / NetIncomeLoss * 100")
    .addFormula("NetIncomeChange%", "NetIncomeLoss - Edgar.lag('NetIncomeLoss', -1) / Edgar.lag('NetIncomeLoss', -1) * 100 ")  
    .addFormula("RevenueChange%", "Edgar.percentChange('Revenue')" )  
    .addFormula("ResearchAndDevelopmentChange%","Edgar.percentChange('ResearchAndDevelopmentExpense')" )
    .removeParameterNames("Revenues","SalesRevenueNet")

val table = values.getTable

CPU times: user 0 ns, sys: 271 µs, total: 271 µs 
Wall Time: 1 s

